In [1]:
%load_ext autoreload
%autoreload 2

import datetime

from llama3.eval import Experiment
from llama3.utils.load_utils import load_config, save_config, setup_log_dir

config = load_config("configs/bayes_eval.yaml")
timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

experiment_log_dir = setup_log_dir(
    config.get("logs_dir", "experiment_logs"),
    timestamp,
    experiment_name=config.get("experiment_name", None),
)

config["experiment_config"]["experiment_log_dir"] = experiment_log_dir
save_config(config.to_dict(), experiment_log_dir + "/config.yaml")

experiment = Experiment(config["experiment_config"])

/home/paperspace/miniconda/envs/posteriors/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.01s/it]
Some weights of BayesLlamaForCausalLM were not initialized from the model checkpoint at Meta-Llama-3-8B and are newly initialized: ['model.bayesian_layers.0.input_layernorm.weight', 'model.bayesian_layers.0.mlp.down_proj.weight', 'model.bayesian_layers.0.mlp.gate_proj.weight', 'model.bayesian_layers.0.mlp.up_proj.weight', 'model.bayesian_layers.0.post_attention_layernorm.weight', 'model.bayesian_layers.0.self_attn.k_proj.weight', 'model.bayesian_layers.0.self_attn.o_proj.weight', 'model.bayesian_layers.0.self_attn.q_proj.weigh

In [2]:
experiment.run(config["dataset_path"])

  0%|          | 1/210 [00:12<42:05, 12.08s/it]